## 1) Import library

In [57]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

Load files

In [58]:
# load Animes
df_animes = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/animes_clean.csv")

In [59]:
# load MiniLM
df_MiniLM = pd.read_json('https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/synopsis_embedding_MiniLM.json')

## 2) LLM - ChatPromptTemplate - Positive and Negative element separator

In [60]:
# Prompt string 
sys_prompt="""
You are a positive and negative element separator. 
Analyze the sentence and separate the elements the user wants (positive) from those they don't want (negative).
Return the response in STRING format and keep only the positive elements.
"""

# Define system prompt
start_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{text}")

])

In [61]:
# Don't forget your API Key
%env MISTRAL_API_KEY=daYzWtTLdhoDl5OnXRDtgqsctIdObhHv

env: MISTRAL_API_KEY=daYzWtTLdhoDl5OnXRDtgqsctIdObhHv


In [62]:
# Let's instanciate a model 
llm = ChatMistralAI(model="mistral-large-latest")

In [69]:
model_llm = start_prompt | llm 

## 3) Natural language input

In [79]:
#input = "I want a Pirates story with crazy adventures"
#input = "I want a sport anime. I love Basketball but I hate Volleyball."
input = "I'm in the mood for a silly, over-the-top comedy with ridiculous characters. I don't want anything serious or emotional"

In [80]:
input_clean = re.sub("[^A-Za-z]+", " ", str(input)).lower()

In [81]:
# Get the response 
response = model_llm.invoke({"text": input_clean})
print(response.content)

"i m in the mood for a silly over the top comedy with ridiculous characters"


In [82]:
input_positive_clean = response.content.strip('"')

## 4) MiniLM Transformer

In [83]:
# pre-trained model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [84]:
input_embedding = model.encode(input_positive_clean)

In [76]:
def search_closest_by_content(content, df, filter):

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df['uid'], 'similarity': similarities})

        # filter by similarity. given_uid exclude
        closest = similarity_df.sort_values(by='similarity', ascending=False).head(5)

        return closest

In [85]:
filter = 'synopsis_embedding'

# call the function and store in DF
result_df = pd.DataFrame(search_closest_by_content(input_embedding, df_MiniLM, filter), columns=['uid','similarity'])
display(result_df)

,uid,similarity
9946,35452,0.555557
9604,17457,0.551202
6452,16371,0.549548
14309,10324,0.541921
10733,7591,0.541921


In [86]:
with pd.option_context('display.max_colwidth', 150):
    display(df_animes[df_animes['uid'].isin(result_df['uid'])])

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
6452,16371,Peeping Life: The Perfect Emotion,"A sequel to Peeping Life comedy anthology, The Perfect Emotion features 10 new animated versions of improvised comedy sketches.","['Slice of Life', 'Comedy']","Apr 16, 2010",10.0,546,10811,9978.0,5.47,https://cdn.myanimelist.net/images/anime/13/44185.jpg,https://myanimelist.net/anime/16371/Peeping_Life__The_Perfect_Emotion
9604,17457,Chingou Muchabee,A goofy comedy series with various stories featuring the same set of characters. \n \n(Source: AniDB),"['Adventure', 'Comedy']","Feb 15, 1971 to Mar 22, 1971",26.0,199,13424,11785.0,6.30,https://cdn.myanimelist.net/images/anime/10/46641.jpg,https://myanimelist.net/anime/17457/Chingou_Muchabee
9946,35452,Hamidashi CoroCoro,Studio DLE and CoroCoro Comic have teamed up to create a series of absurd comedy shorts whose characters are based on reader questionnaires.,"['Comedy', 'Parody']","Jul 14, 2016 to Dec 15, 2016",6.0,94,15453,12187.0,6.06,https://cdn.myanimelist.net/images/anime/10/85457.jpg,https://myanimelist.net/anime/35452/Hamidashi_CoroCoro
10733,7591,Tono to Issho,Comedy parodying famous samurai generals and historical events.,"['Comedy', 'Historical', 'Parody', 'Samurai', 'Seinen']","Mar 25, 2010",1.0,3390,6711,5359.0,6.65,https://cdn.myanimelist.net/images/anime/5/18878.jpg,https://myanimelist.net/anime/7591/Tono_to_Issho
14309,10324,Tono to Issho: Gantai no Yabou,Comedy parodying famous samurai generals and historical events.,"['Comedy', 'Historical', 'Parody', 'Samurai', 'Seinen']","Apr 5, 2011 to Jun 21, 2011",12.0,2520,7404,5951.0,6.53,https://cdn.myanimelist.net/images/anime/10/28398.jpg,https://myanimelist.net/anime/10324/Tono_to_Issho__Gantai_no_Yabou
